In [1]:
library(tidyverse)
library(rjson)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.0     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Read in the downloaded data set

In [2]:
nlcd_dat_path = list.files("../data/dataset/nlcd_nature_deprived", full.names = TRUE)
nlcd_raw_dat = read_csv(nlcd_dat_path)
head(nlcd_raw_dat)


── Column specification ────────────────────────────────────────────────────────
cols(
  GEOID10_TRACT = col_double(),
  `Does the tract have at least 35 acres in it?` = col_logical(),
  `Share of the tract's land area that is covered by impervious surface as a percent` = col_double(),
  `Share of the tract's land area that is covered by impervious surface or cropland as a percent` = col_double(),
  `Share of the tract's land area that is covered by cropland as a percent` = col_double()
)




GEOID10_TRACT,Does the tract have at least 35 acres in it?,Share of the tract's land area that is covered by impervious surface as a percent,Share of the tract's land area that is covered by impervious surface or cropland as a percent,Share of the tract's land area that is covered by cropland as a percent
<dbl>,<lgl>,<dbl>,<dbl>,<dbl>
27139080202,TRUE,37.3610,37.5887,0.227730
27139080204,TRUE,26.8979,27.6588,0.760943
27139080100,TRUE,39.4516,39.9349,0.483341
27139080302,TRUE,18.2605,37.6391,19.378600
27139080400,TRUE,47.4381,48.4268,0.988653
27139080500,TRUE,42.8099,43.9226,1.112750


In [3]:
impervious_in = pull(nlcd_raw_dat,4) ## impervious surfaces or cropland
sum(impervious_in[is.na(impervious_in)]) ## check for missing data
length(impervious_in)
sum(impervious_in >= quantile(impervious_in, .9)) ## calculate the number at or above 90th percentile

[1] 0

[1] 72539

[1] 7254

There are 7254 tracts in the read in that have an impervious surface percent above the 90%

 Then the data gets pulled into an ETL process and combined into one data set where scores are calculated. Lets check that output.


In [4]:
score_data_full_path = list.files("../data/score/csv/full", full.names = TRUE)[2]
score_dat_full = read_csv(score_data_full_path)


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  GEOID10_TRACT = col_character(),
  `Does the tract have at least 35 acres in it?` = col_logical(),
  `Contains agricultural value` = col_logical(),
  `Names of Tribal areas within Census tract` = col_character(),
  `Percent individuals age 25 or over with less than high school degree in 2009` = col_logical(),
  `Percentage households below 100% of federal poverty line in 2009` = col_logical(),
  `Unemployment (percent) in 2009` = col_logical(),
  `Total population in 2009` = col_logical(),
  `Number of Tribal areas within Census tract for Alaska` = col_logical(),
  `Number of Tribal areas within Census tract` = col_logical(),
  `Median household income as a percent of territory median income in 2009` = col_logical(),
  `Is there at least one abandoned mine in this census tract?` = col_logical(),
  `Income data has been estimated based on neighbor income` = col_logical(),

In [5]:
score_full_cols = colnames(score_dat_full)
score_full_cols
impervious_cols = score_full_cols[grepl("impervious", score_full_cols)]
score_dat_impervious = score_dat_full[,impervious_cols ]
head(score_dat_impervious)

[1] "GEOID10_TRACT"                                                                                                                                                    
  [2] "Does the tract have at least 35 acres in it?"                                                                                                                     
  [3] "Contains agricultural value"                                                                                                                                      
  [4] "Names of Tribal areas within Census tract"                                                                                                                        
  [5] "Housing burden (percent)"                                                                                                                                         
  [6] "Share of homes with no kitchen or indoor plumbing (percent)"                                                                                                      
  [7] "Total population"                                                                                                                                                 
  [8] "Median household income (% of state median household income)"                                                                                                     
  [9] "Current asthma among adults aged greater than or equal to 18 years"                                                                                               
 [10] "Coronary heart disease among adults aged greater than or equal to 18 years"                                                                                       
 [11] "Cancer (excluding skin cancer) among adults aged greater than or equal to 18 years"                                                                               
 [12] "Current lack of health insurance among adults aged 18-64 years"                                                                                                   
 [13] "Diagnosed diabetes among adults aged greater than or equal to 18 years"                                                                                           
 [14] "Physical health not good for greater than or equal to 14 days among adults aged greater than or equal to 18 years"                                                
 [15] "Percent of individuals < 100% Federal Poverty Line"                                                                                                               
 [16] "Percent of individuals < 150% Federal Poverty Line"                                                                                                               
 [17] "Percent of individuals below 200% Federal Poverty Line"                                                                                                           
 [18] "Area Median Income (State or metropolitan)"                                                                                                                       
 [19] "Median household income in the past 12 months"                                                                                                                    
 [20] "Energy burden"                                                                                                                                                    
 [21] "FEMA Risk Index Expected Annual Loss Score"                                                                                                                       
 [22] "Urban Heuristic Flag"                                                                                                                                             
 [23] "Air toxics cancer risk"                                                                                                                                           
 [24] "Respiratory hazard index"                                                            

Share of the tract's land area that is covered by impervious surface or cropland as a percent,Share of the tract's land area that is covered by impervious surface or cropland as a percent (percentile),Greater than or equal to the 90th percentile for share of the tract's land area that is covered by impervious surface or cropland as a percent,Greater than or equal to the 90th percentile for share of the tract's land area that is covered by impervious surface or cropland as a percent and is low income?
<dbl>,<dbl>,<lgl>,<lgl>
21.2475,0.2685590,FALSE,FALSE
42.1254,0.5569694,FALSE,FALSE
18.3748,0.2323578,FALSE,FALSE
37.9230,0.4961469,FALSE,FALSE
51.4835,0.6894912,FALSE,FALSE
54.5483,0.7298005,FALSE,FALSE


In [6]:
# check base score.
sum(is.na(score_dat_impervious[,1]))
nrow(score_dat_impervious) - length(impervious_in) ## the missing tracts were initiated after the join. they aren't in the original data set. Its likely because of the CONUS issue. 
sum(score_dat_impervious[,1] >= quantile(score_dat_impervious[,1], .9, na.rm = TRUE), na.rm = TRUE) ## calculate the number at or above 90th percentile

[1] 1595

[1] 1595

[1] 7254

In [7]:
# Check the calculated percentile
sum(score_dat_impervious[, 2] >= .9, na.rm = TRUE)
## the calculated percentile lines up

[1] 7254

In [8]:
# Check in on the TRUE/FALSE Flag
sum(score_dat_impervious[,3] == TRUE)
## okay, the flag here is correct as well. So where does it go wrong?

[1] 7254

Next, the data gets post processed to add in demographic information about the tracts

In [9]:
score_post_full_path = list.files("../data/score/downloadable", pattern = ".csv", full.names = TRUE)[3]
score_post = read_csv(score_post_full_path)


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  `Census tract 2010 ID` = col_character(),
  `County Name` = col_character(),
  `State/Territory` = col_character(),
  `Identified as disadvantaged without considering neighbors` = col_logical(),
  `Identified as disadvantaged based on neighbors and relaxed low income threshold only` = col_logical(),
  `Identified as disadvantaged due to tribal overlap` = col_logical(),
  `Identified as disadvantaged` = col_logical(),
  `Is low income?` = col_logical(),
  `Income data has been estimated based on geographic neighbor income` = col_logical(),
  `Greater than or equal to the 90th percentile for expected agriculture loss rate and is low income?` = col_logical(),
  `Greater than or equal to the 90th percentile for expected building loss rate and is low income?` = col_logical(),
  `Greater than or equal to the 90th percentile for expected population loss rate and is low income?`

In [10]:
score_post_cols = colnames(score_post)
score_post_impervious_cols = score_post_cols[grepl("impervious", score_post_cols)]
score_post_impervious = score_post[,score_post_impervious_cols]
head(score_post_impervious)

Greater than or equal to the 90th percentile for share of the tract's land area that is covered by impervious surface or cropland as a percent and is low income?,Greater than or equal to the 90th percentile for share of the tract's land area that is covered by impervious surface or cropland as a percent,Share of the tract's land area that is covered by impervious surface or cropland as a percent,Share of the tract's land area that is covered by impervious surface or cropland as a percent (percentile)
<lgl>,<lgl>,<dbl>,<dbl>
FALSE,FALSE,760,10
FALSE,FALSE,2058,26
FALSE,FALSE,2353,29
FALSE,FALSE,2333,29
FALSE,FALSE,3955,51
FALSE,FALSE,1937,24


In [11]:
# check base score. That still came through
sum(is.na(score_post_impervious[,3]))
nrow(score_post_impervious) - length(impervious_in) 
sum(score_post_impervious[,3] >= quantile(score_post_impervious[,3], .9, na.rm = TRUE), na.rm = TRUE) ## calculate the number at or above 90th percentile

[1] 1595

[1] 1595

[1] 7258

I think there is likely just a rounding error that made that 7258 instead of 7254

In [12]:
sum(score_post_impervious[, 4] >= 90, na.rm = TRUE)

[1] 7254

In [13]:
sum(score_post_impervious[,2] == TRUE)

[1] 7254

Take a look at the tiles themselves. 

In [14]:
score_tiles_path = list.files("../data/score/csv/tiles", pattern = ".csv", full.names = TRUE)
score_tiles = read_csv(score_tiles_path)


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_logical(),
  GTF = col_character(),
  SF = col_character(),
  CF = col_character(),
  DF_PFS = col_double(),
  AF_PFS = col_double(),
  HDF_PFS = col_double(),
  DSF_PFS = col_double(),
  EBF_PFS = col_double(),
  EALR_PFS = col_double(),
  EBLR_PFS = col_double(),
  EPLR_PFS = col_double(),
  HBF_PFS = col_double(),
  LLEF_PFS = col_double(),
  LIF_PFS = col_double(),
  LMI_PFS = col_double(),
  PM25F_PFS = col_double(),
  HSEF = col_double(),
  P100_PFS = col_double(),
  P200_I_PFS = col_double(),
  LPF_PFS = col_double()
  # ... with 30 more columns
)
ℹ Use `spec()` for the full column specifications.


Warning message:
“513 parsing failures.
 row         col           expected actual                              file
1182 TA_COUNT_AK 1/0/T/F/TRUE/FALSE    1.0 '../data/score/csv/tiles/usa.csv'
1183 TA_COUNT_AK 1/0/T/F/TRUE/FALSE    4.0 '../data/score/csv/tiles/usa.csv'
1184 TA_CO

In [15]:
score_tiles %>% select(IS_PFS, IS_ET) %>%
summarise(
    IS_PFS = sum(IS_PFS >=.9, na.rm = TRUE) 
    , IS_ET = sum(IS_ET) # okay, this came through correctly this time. 
)

IS_PFS,IS_ET
<int>,<int>
7254,7254


In [16]:
score_mapping_path = list.files("../data/score/csv/tiles", pattern = "json", full.names = TRUE)
 score_mapping = fromJSON(file = score_mapping_path)

In [17]:
tibble(
    abbrev = names(score_mapping)
    , names = flatten(score_mapping )
    ) %>%
filter(grepl("impervious", names))

abbrev,names
<chr>,<named list>
IS_PFS,Share of the tract's land area that is covered by impervious surface or cropland as a percent (percentile)
IS_ET,Greater than or equal to the 90th percentile for share of the tract's land area that is covered by impervious surface or cropland as a percent


Next, we have the Json with the shape files. And the problem propogates into the .json files. 

In [18]:
high_res_json_path = list.files("../data/score/geojson", pattern = "high", full.names = TRUE)
high_res_json <- fromJSON(file = high_res_json_path)

In [19]:
glimpse(high_res_json)

List of 3
 $ type    : chr "FeatureCollection"
 $ crs     :List of 2
  ..$ type      : chr "name"
  ..$ properties:List of 1
  .. ..$ name: chr "urn:ogc:def:crs:OGC:1.3:CRS84"
 $ features:List of 74134
  ..$ :List of 3
  .. ..$ type      : chr "Feature"
  .. ..$ properties:List of 123
  .. ..$ geometry  :List of 2
  ..$ :List of 3
  .. ..$ type      : chr "Feature"
  .. ..$ properties:List of 123
  .. ..$ geometry  :List of 2
  ..$ :List of 3
  .. ..$ type      : chr "Feature"
  .. ..$ properties:List of 123
  .. ..$ geometry  :List of 2
  ..$ :List of 3
  .. ..$ type      : chr "Feature"
  .. ..$ properties:List of 123
  .. ..$ geometry  :List of 2
  ..$ :List of 3
  .. ..$ type      : chr "Feature"
  .. ..$ properties:List of 123
  .. ..$ geometry  :List of 2
  ..$ :List of 3
  .. ..$ type      : chr "Feature"
  .. ..$ properties:List of 123
  .. ..$ geometry  :List of 2
  ..$ :List of 3
  .. ..$ type      : chr "Feature"
  .. ..$ properties:List of 123
  .. ..$ geometry  :List of 2


We want IS_PFS and IS_ET

In [20]:
high_res_json$type
high_res_json$features[[1]]$properties$IS_PFS
high_res_json$features[[1]]$properties$IS_ET

[1] "FeatureCollection"

[1] 0.26

[1] FALSE

In [21]:
impervious_json = map(
    # 1:10, 
     1:length(high_res_json$features), 
    ~ tibble(
     GEOID =  high_res_json$features[[.x]]$properties$GEOID
   , IS_PFS = as.numeric(high_res_json$features[[.x]]$properties$IS_PFS)
   , IS_ET = high_res_json$features[[.x]]$properties$IS_ET
        )
    )   

In [22]:
impervious_json_df <- bind_rows(impervious_json)
head(impervious_json_df)

GEOID,IS_PFS,IS_ET
<chr>,<dbl>,<lgl>
01073001100,0.26,FALSE
01073001400,0.55,FALSE
01073002000,0.23,FALSE
01073003802,0.49,FALSE
01073004000,0.68,FALSE
01073005101,0.72,FALSE


In [23]:
sum(impervious_json_df$IS_PFS >= .9)
sum(impervious_json_df$IS_ET)


[1] 7254

[1] 7254

In [24]:
impervious_json_df %>% filter(IS_PFS >= .9, IS_ET == FALSE) %>% head() # This is correct!

GEOID,IS_PFS,IS_ET
<chr>,<dbl>,<lgl>


    okay, how did we get that the IS_ET is false when IS_PFS >= .9?. This is now fixed


In [27]:
impervious_json_df %>% filter(GEOID == "11001004701")

GEOID,IS_PFS,IS_ET
<chr>,<dbl>,<lgl>
11001004701,0.95,TRUE


In [45]:
library(osmextract)
library(sf)

Linking to GEOS 3.8.1, GDAL 3.1.4, PROJ 6.3.1



In [54]:
tile_path = list.files("../data/score/tiles/low/0/0", full.names = TRUE)[1]
tile_path

[1] "../data/score/tiles/low/0/0/0.gpkg"

In [56]:
tile_read = oe_read(tile_path)

ERROR: Error: Cannot open "/Users/samuelpowers/Documents/USDS/justice40-tool/data/data-pipeline/data_pipeline/data/score/tiles/low/0/0/0.gpkg"; The source could be corrupt or not supported. See `st_drivers()` for a list of supported formats.


In [44]:
tile_read

ERROR: Error in eval(expr, envir, enclos): object 'tile_read' not found
